In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import cv2
import pydot

In [2]:
#load_dataset function to load the data and resize the images to 50x50
def load_dataset(directory):
  images = []
  labels = []
  for idx, label in enumerate(uniq_labels):
    for file in os.listdir(directory + '/'+label):
      filepath = directory +'/'+ label + "/" + file
      img = cv2.resize(cv2.imread(filepath),(50,50))
      images.append(img)
      labels.append(idx)
  images = np.asarray(images)
  labels = np.asarray(labels)
  return images, labels

In [3]:
#loading_dataset into X_pre and Y_pre
data_dir = "image3/train/"
uniq_labels = sorted(os.listdir(data_dir))
X_pre, Y_pre = load_dataset(data_dir)
print(X_pre.shape, Y_pre.shape)

(4485, 50, 50, 3) (4485,)


In [4]:
#spliting dataset into 80% train, 10% validation and 10% test data
X_train, X_test, Y_train, Y_test = train_test_split(X_pre, Y_pre, test_size = 0.8)
X_test, X_eval, Y_test, Y_eval = train_test_split(X_test, Y_test, test_size = 0.5)


In [5]:
# converting Y_tes and Y_train to One hot vectors using to_categorical
# example of one hot => '1' is represented as [0. 1. 0. . . . . 0.]
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)
Y_eval = to_categorical(Y_eval)
X_train = X_train / 255.
X_test = X_test/ 255.
X_eval = X_eval/ 255.

In [6]:
# building our model
model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation ='relu', input_shape=(50,50,3)),
        tf.keras.layers.Conv2D(16, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(16, (3,3), activation ='relu'),
        tf.keras.layers.MaxPool2D((2,2)),
        tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
        tf.keras.layers.MaxPool2D((2,2)),
        tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(8, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 16)        448       
                                                                 
 conv2d_1 (Conv2D)           (None, 46, 46, 16)        2320      
                                                                 
 conv2d_2 (Conv2D)           (None, 44, 44, 16)        2320      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 16)       0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, 20, 20, 32)        4640      
                                                                 
 conv2d_4 (Conv2D)           (None, 18, 18, 32)        9248      
                                                        

In [7]:
#compiling the model
#default batch size 32
#default learning rate is 0.001
model.compile(optimizer = 'adam',
               loss = 'categorical_crossentropy',
               metrics=['accuracy'],)

In [8]:
#start training(fitting) the data
history = model.fit(X_train, Y_train, epochs=100, verbose=1,
                validation_data=(X_eval, Y_eval))

Epoch 1/100
29/29 [==============================] - 5s 107ms/step - loss: 2.0801 - accuracy: 0.1449 - val_loss: 2.0786 - val_accuracy: 0.1377
Epoch 2/100
29/29 [==============================] - 2s 85ms/step - loss: 1.9738 - accuracy: 0.2564 - val_loss: 1.7940 - val_accuracy: 0.3138
Epoch 3/100
29/29 [==============================] - 3s 94ms/step - loss: 1.6067 - accuracy: 0.4448 - val_loss: 1.5376 - val_accuracy: 0.4548
Epoch 4/100
29/29 [==============================] - 3s 89ms/step - loss: 1.3127 - accuracy: 0.5474 - val_loss: 1.2942 - val_accuracy: 0.5424
Epoch 5/100
29/29 [==============================] - 3s 100ms/step - loss: 1.0755 - accuracy: 0.6410 - val_loss: 1.5408 - val_accuracy: 0.5546
Epoch 6/100
29/29 [==============================] - 3s 91ms/step - loss: 0.8910 - accuracy: 0.7068 - val_loss: 0.9341 - val_accuracy: 0.6934
Epoch 7/100
29/29 [==============================] - 3s 97ms/step - loss: 0.6716 - accuracy: 0.7882 - val_loss: 0.8181 - val_accuracy: 0.7553
Epoc

In [9]:
#testing
model.evaluate(X_test, Y_test)

57/57 [==============================] - 1s 14ms/step - loss: 0.7554 - accuracy: 0.9220


[0.7553585767745972, 0.9219620823860168]

In [10]:
model.save("model_save/test_avec_crop_mediapipe.h5")